In [72]:
#!pip install -r requirements.txt

In [10]:
import pandas as pd

In [8]:
df = pd.read_csv('./data/lyrics.csv', sep=';', encoding="utf-8")
df.shape

(13405, 7)

1. Remoção dos títulos das músicas que contenham os caracteres '()' 

In [9]:
df['title'].replace("\((.*?)\)", '', regex=True, inplace=True)



2. Remoção das músicas duplicadas a partir do título

In [10]:
df['title'] = df['title'].str.lower()
df['title'] = df['title'].str.strip()

titles = [i for i in df['title']]

duplicates = [i for i in titles if titles.count(i) > 1]
print(f'Quantidade de duplicatas: {len(duplicates)}')

Quantidade de duplicatas: 3484


In [11]:
df = df[~df['title'].isin(duplicates)]
print(f'Quantidade de músicas após remoção das duplicatas: {df.id.count()}')

Quantidade de músicas após remoção das duplicatas: 9921


3. Remoção das músicas sem data

In [12]:
df = df[df['year'] > 0]

print(f'Quantidade de músicas com data: {df.id.count()}')

Quantidade de músicas com data: 8176


4. Remoção dos registros sem letras

In [13]:
df = df[~df['lyric'].isnull()]
print(f'Quantidade de registros com letras: {df.lyric.count()}')

Quantidade de registros com letras: 8133


In [13]:
df = pd.read_csv('./data/clean_lyrics.csv')

In [ ]:
#removendo ruídos das letras
df['clean_lyric'] = df['lyric'].apply(clean_lyrics)

In [32]:
#gerando decadas
df['decade'] = df['year'].apply(decade)
#df.tail()

Foi identificado que algumas letras que apresentavam 
<br>
sujeiras foram removidas pelo processo de limpeza.
<br> 
Devido a isso foi necessário repetir a limpeza dos 
<br>
registros que estavam com as letras das músicas nulas.

In [42]:
#removendo letras nulas
df = df[~df['clean_lyric'].isnull()]

In [43]:
# gerando colunas de palavras únicas
df['unique_words'] = df['clean_lyric'].apply(unique)
df.head()

,id,title,artist,composer,album,year,lyric,decade,clean_lyric,unique_words,words
0,2360,João Teimoso,Marília Batista,Noel Rosa / Marilia Batista.,Poeta da Vila,1953,Tenho mais o que fazer Não discuto com teimoso...,50,discuto teimoso perder precioso viver joão tei...,"{precioso, viver, boas, coroas, nome, cismo, f...","[discuto, teimoso, perder, precioso, viver, jo..."
1,2361,Morena Sereia,Marília Batista,José Maria De Abreu 1936 / Noel Rosa.,Poeta da Vila,1953,Morena sereia Que à beira-mar não passeia Que ...,50,morena sereia beiramar passeia senta praia dei...,"{tufão, lindos, esqueci, nome, morena, derruba...","[morena, sereia, beiramar, passeia, senta, pra..."
2,2362,"Parabéns, Guanabara",Jackson do Pandeiro,NaN,Jackson do Pandeiro - 1955,1955,"Rio, querida Guanabara Eu sou gente também Ace...",50,rio querida guanabara gente aceito parabéns pa...,"{adoram, guanabara, vim, noel, aceito, bolo, c...","[rio, querida, guanabara, gente, aceito, parab..."
3,2363,O Bom Xaxador,Jackson do Pandeiro,NaN,Jackson do Pandeiro - 1955,1955,Depois do baião foi que veio o forró Depois do...,50,baião veio forró forró veio rojão pisada chão ...,"{rojão, veio, xaxado, vim, chão, forró, xaxa, ...","[baião, veio, forró, forró, veio, rojão, pisad..."
4,2364,Cheguei Agora,Jackson do Pandeiro,NaN,Jackson do Pandeiro - 1955,1955,Beira mar beira do rio Ê ê beira mar Cheguei a...,50,beira mar beira rio ê ê beira mar cheguei beir...,"{candomblé, bahia, ê, café, coco, carioca, sam...","[beira, mar, beira, rio, ê, ê, beira, mar, che..."


In [ ]:
# transformando letras em lista de palavras
df['words'] = df['clean_lyric'].apply(to_words)
df.head()

In [37]:
#Passando os titulos para o padrão CamelCase
df['title'] = df['title'].str.title()

In [15]:
#armazenando
df.to_csv('./data/clean_lyrics.csv', index=False)

### Funções utilitárias

In [2]:
#função para criar decadas
decade = lambda year: int(((((year - (year % 10)) / 10) % 10) * 10))

In [3]:
#função para retornar palavras em formato de lista
to_words = lambda text: text.split()

In [4]:
#função para retornar palavras unicas
unique = lambda text: set(text.split())

In [5]:
#carregando lista de acordes para remoção dentro das letras
chords_df = pd.read_csv('./data/chords.txt')
chords_dicty = chords_df.to_dict()
chords = set()

for i in chords_dicty['chord']:
    chords.add(chords_dicty['chord'][i].strip())
    
#função para remoção de acordes dentro de um texto
def remove_chords(text):
    txt_list = text.split()
    return (' ').join(
            word.strip()
            for word in txt_list
            if word.strip() not in chords
        )    

In [16]:
import string
import re
import spacy

nlp = spacy.load('pt_core_news_lg')

def clean_lyrics(text):
  txt = ''
  #removendo acordes
  txt = remove_chords(text)
  #removendo tags de repetição 2x, 4x, etc
  txt = txt.replace('[', '(').replace(']', ')')
  txt = re.sub(r'\([\d]x\)', '', txt)
  #removendo números
  txt = re.sub(r'[\d]', '', txt)
  #removendo palavras de orientação musical
  txt = re.sub(r'\((.*?)\)', '', txt)  
  #removendo pontuacao
  txt = txt.translate(str.maketrans("", "", string.punctuation))
  #caracteres em minusculo   
  txt = txt.lower()
  #remove stop words
  doc = nlp(txt)
  txt = ' '.join(token.text 
                  for token in doc 
                  if not token.is_stop
                  and not token.is_digit
                )
  
  return txt